In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
import matplotlib
from tqdm  import tqdm
headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36'
}

# 所公告的研發計畫

In [2]:
end = 7
plans1 = []
for i in range(1,end):
    res = requests.get(f'https://tiip.itnet.org.tw/news_list.php?m=1&page={i}')

    soup = BeautifulSoup(res.text, "html.parser")
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(res.content, 'lxml')

    # Find all rows in the table
    rows = soup.find_all("tr")[2:]  # Skip the header row

    # Extract the necessary information: 公佈時間, 標題, and URL
    for row in rows:
        columns = row.find_all("td")
        pub_date = columns[1].text.strip()
        title = columns[2].text.strip()
        link = 'https://tiip.itnet.org.tw/'+columns[2].find("a")["href"]
        plans1.append({"公佈時間": pub_date, "標題": title, "連結": link})

# 所有核定名單

In [ ]:
# '''version1'''
# end = 7
# plans2 = []
# for i in range(1,end):
#     res = requests.get(f'https://tiip.itnet.org.tw/news_list.php?m=3&page={i}')

#     # soup = BeautifulSoup(res.text, "html.parser")
#     # Parse the HTML content using BeautifulSoup
#     soup = BeautifulSoup(res.content, 'lxml')

#     # Find all rows in the table
#     rows = soup.find_all("tr")[2:]  # Skip the header row

#     # Extract the necessary information: 公佈時間, 標題, and URL
#     for row in rows:
#         columns = row.find_all("td")
#         pub_date = columns[1].text.strip()
#         title = columns[2].text.strip()
#         link = 'https://tiip.itnet.org.tw/'+columns[2].find("a")["href"]
#         plans2.append({"公佈時間": pub_date, "標題": title, "連結": link})

In [23]:
# '''version2'''
end = 90
plans2 = []
for i in tqdm(range(1,end)):
    try:
        res = requests.get(f'https://tiip.itnet.org.tw/achievementList.php?year=&kw=&page={i}')
    except:
        break

    # soup = BeautifulSoup(res.text, "html.parser")
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(res.content, 'lxml')

    # Find all rows in the table
    rows = soup.find_all("tr")[2:]  # Skip the header row

    # Extract the necessary information: 公佈時間, 標題, and URL
    for row in rows:
        columns = row.find_all("td")
        company_name = columns[2].text.strip()
        plans_name = columns[3].text.strip()
        date = columns[4].text.strip()
        # link = 'https://tiip.itnet.org.tw/'+columns[2].find("a")["href"]
        plans2.append({"公司名稱": company_name, "計畫名稱": plans_name, "核定日期": date})

100%|██████████| 89/89 [02:08<00:00,  1.45s/it]


# 各計畫核定公司

In [4]:
plans2 = pd.DataFrame(plans2)

In [ ]:
plans = []
for i in tqdm(range(0,len(plans2))):    
    res = requests.post(plans2['連結'][i])
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(res.content, 'lxml')

    # 計畫類別
    plans_type = soup.find_all('title')[0].text
    # Extract the company names from the table
    rows = soup.find_all("tr")[2:]  # Skip the header row
    for row in rows:
        columns = row.find_all("td")
        company_name = columns[2].text.strip()
        plans_name = columns[3].text.strip()
        date = columns[4].text.strip()
        plans.append({"計畫類別": plans_type,"公司名稱": company_name, "計畫名稱": plans_name, "核定日期": date})

In [6]:
tmp = pd.DataFrame(plans)
tmp.to_excel('4_政府推動計畫名單.xlsx', index = False)